In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#df = pd.read_csv(r"/content/drive/MyDrive/dataset/sentiment_dataset.csv")
file_path = "/content/drive/MyDrive/dataset/sentiment_dataset.csv"

# column names (since the file has no headers)
columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

df = pd.read_csv(file_path, encoding='latin-1', names=columns)

# drop the unnececary columns
df = df[['text', 'target']]

print(f"Shape: {df.shape}")
df.head()

In [ ]:
# define the english stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
# function for cleaning up text (e.g. removing the @ tags from the tweets)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [ ]:
df['cleaned_text'] = df['text'].apply(clean_text)

# convert targets: 0 -> 0 (negative), 4 -> 1 (positive)
df['target'] = df['target'].apply(lambda x: 0 if x == 0 else 1)

df[['cleaned_text', 'target']].head()

# GRU Model